In [6]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [7]:
# Import libraries
from fastai import *
from fastai.vision import *

# Set seed
np.random.seed(7)
SEED = 7

In [8]:
# Check if data is available
path = Config.data_path()/'planet'
path.mkdir(parents=True, exist_ok=True)
path

PosixPath('/home/jupyter/.fastai/data/planet')

In [9]:
# Look at the files available
path.ls()

[PosixPath('/home/jupyter/.fastai/data/planet/train_v2.csv'),
 PosixPath('/home/jupyter/.fastai/data/planet/train_v2.csv.zip'),
 PosixPath('/home/jupyter/.fastai/data/planet/__MACOSX'),
 PosixPath('/home/jupyter/.fastai/data/planet/test-jpg.tar.7z'),
 PosixPath('/home/jupyter/.fastai/data/planet/train-jpg'),
 PosixPath('/home/jupyter/.fastai/data/planet/test-jpg-additional'),
 PosixPath('/home/jupyter/.fastai/data/planet/models'),
 PosixPath('/home/jupyter/.fastai/data/planet/train-jpg.tar.7z'),
 PosixPath('/home/jupyter/.fastai/data/planet/test-jpg-additional.tar.7z'),
 PosixPath('/home/jupyter/.fastai/data/planet/test-jpg'),
 PosixPath('/home/jupyter/.fastai/data/planet/test-jpg-additional.tar')]

Now, let's look at how the labels for the images are provided to us

In [10]:
df = pd.read_csv(path/'train_v2.csv')
df.head()

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


In [11]:
src = (ImageItemList
       .from_csv(path, 'train_v2.csv', folder='train-jpg', suffix='.jpg')
       .random_split_by_pct(0.2, seed=SEED)
       .label_from_df(sep=' ')
       .add_test_folder('test-jpg'))
src

LabelLists;

Train: LabelList
y: MultiCategoryList (32384 items)
[MultiCategory haze;primary, MultiCategory agriculture;clear;primary;water, MultiCategory clear;primary, MultiCategory clear;primary, MultiCategory agriculture;clear;habitation;primary;road]...
Path: /home/jupyter/.fastai/data/planet
x: ImageItemList (32384 items)
[Image (3, 256, 256), Image (3, 256, 256), Image (3, 256, 256), Image (3, 256, 256), Image (3, 256, 256)]...
Path: /home/jupyter/.fastai/data/planet;

Valid: LabelList
y: MultiCategoryList (8095 items)
[MultiCategory clear;primary, MultiCategory clear;primary, MultiCategory clear;primary, MultiCategory agriculture;bare_ground;clear;habitation;primary;road;water, MultiCategory agriculture;blooming;clear;cultivation;primary;slash_burn]...
Path: /home/jupyter/.fastai/data/planet
x: ImageItemList (8095 items)
[Image (3, 256, 256), Image (3, 256, 256), Image (3, 256, 256), Image (3, 256, 256), Image (3, 256, 256)]...
Path: /home/jupyter/.fastai/data/planet;

Test: La

In [12]:
tfms = get_transforms(flip_vert=True, max_zoom=1.05, max_lighting=0.1, max_warp=0.0)

In [13]:
def get_data(BS, size):
    '''
    BS: batch_size
    size: image_size
    '''
    data = (src.transform(tfms, size=size)
            .databunch(bs=BS)
            .normalize(imagenet_stats))

In [14]:
data = get_data(32,128)